# Preprocess data

## 1. Extract and clean data

In [1]:
import pandas as pd

from geopy import distance

/home/liangwei/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/liangwei/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
# Load routes
routes = pd.read_csv("./data/routes.dat",
                      names = ["airline",
                              "airline_id",
                              "source_airport",
                              "source_airport_id",
                              "destination_airport",
                              "destination_airport_id",
                              "codeshare",
                              "stops",
                              "equipment"])

In [6]:
routes.head()

,airline,airline_id,source_airport,source_airport_id,destination_airport,destination_airport_id,codeshare,stops,equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2


In [7]:
# Load airports 
airports = pd.read_csv("data/airports.dat",
                       names = ["airport_id", "Name", "City", "Country", "IATA",
                                "ICAO", "Latitude", "Longitude",
                                "Altitude", "Timezone", "DST",
                                "TzDatabase", "Type", "Source"])

In [535]:
# Add continent info to airports
airports["Continent"] = airports["TzDatabase"].str.split("/").str[0]

In [538]:
# Display samples of airports
airports.head()

,airport_id,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TzDatabase,Type,Source,Continent
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports,Pacific
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports,Pacific
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports,Pacific
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports,Pacific
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports,Pacific


In [420]:
# Clean the data
airports_routes = []
for i in list(set(routes['source_airport_id']) | set(routes['destination_airport_id'])):
    try:
        airports_routes.append(int(i))
    except:
        continue

airports_airports = list(airports['airport_id'])

airports_cleaned = list(set(airports_routes) & set(airports_airports))
airports_cleaned_str = [str(i) for i in airports_cleaned]

routes_cleaned = routes[routes['source_airport_id'].isin(airports_cleaned_str) & routes['destination_airport_id'].isin(airports_cleaned_str)]

airports_cleaned = airports[airports['airport_id'].isin(airports_cleaned)]

In [421]:
print(f'The original number of routes is {len(routes)}, and the reduced number of routes is {len(routes_cleaned)}')

The original number of routes is 67663, and the reduced number of routes is 66771


In [422]:
print(f'The original number of airports is {len(airports)}, and the reduced number of airports is {len(airports_cleaned)}')

The original number of airports is 7698, and the reduced number of airports is 3221


## Discover incorrect airports by checking timezones


In [548]:
# Read in country timezone dataframe
country_timezone_df = pd.read_csv("./data/timezones.csv", encoding='latin-1')

In [549]:
country_timezone_df["Continent"] = country_timezone_df["Time Zone"].str.split("/").str[0]

In [550]:
# Generate error candidates by finding the airport whose country does not match the timezones
error_candidates = airports[airports.apply(lambda row: row["TzDatabase"].split("/")[0] not in
                        country_timezone_df[country_timezone_df["Country Name"] ==
                                            row["Country"]]["Continent"].unique().tolist(),
                       axis = 1)]

# By checking the error candidates, we obtain several airport entries which are wrong
# Two instance are listed below
error_candidates[error_candidates["airport_id"].apply(lambda entry: entry in [8782, 5613])]

,airport_id,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TzDatabase,Type,Source,Continent
2376,5613,Los Alamitos Army Air Field,Solwesi,Zambia,\N,KSLI,33.790001,-118.052002,32,-8,U,America/Los_Angeles,airport,OurAirports,America
3157,8782,Cooma Hospital Helipad,Yuzhno-Kurilsk,Russia,\N,YXCM,-36.242155,149.130147,2656,10,N,Australia/Sydney,airport,OurAirports,Australia


In [423]:
# Save the cleaned data
routes_cleaned.to_csv('data/routes_cleaned.csv')
airports_cleaned.to_csv('data/airports_cleaned.csv')

## 2. Create graph

In [525]:
routes = pd.read_csv("./data/routes_cleaned.csv")
airports = pd.read_csv("./data/airports_cleaned.csv")

In [527]:
airports[airports["airport_id"] == 2965]

,Unnamed: 0,airport_id,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TzDatabase,Type,Source
1421,2810,2965,Sochi International Airport,Sochi,Russia,AER,URSS,43.449902,39.9566,89,3,N,Europe/Moscow,airport,OurAirports


### 2.1 Create nodes

In [425]:
import numpy as np

In [426]:
# Get source and destination airports from routes
src_airports = routes[["source_airport_id", "source_airport"]]
src_airports.columns = ["airport_id", "airport"]
dst_airports = routes[["destination_airport_id", "destination_airport"]]
dst_airports.columns = ["airport_id", "airport"]

# Create nodes from airports
nodes = pd.concat([src_airports, dst_airports], axis = 0).drop_duplicates()
nodes.replace(to_replace = "\\N", value = np.NaN, inplace = True)
nodes.dropna(axis = 0, inplace = True)
nodes["airport_id"] = nodes["airport_id"].apply(int)

In [427]:
airports.columns

Index(['Unnamed: 0', 'airport_id', 'Name', 'City', 'Country', 'IATA', 'ICAO',
       'Latitude', 'Longitude', 'Altitude', 'Timezone', 'DST', 'TzDatabase',
       'Type', 'Source'],
      dtype='object')

In [428]:
# Add latitudes, longitudes and continents to nodes
nodes = nodes.merge(right = airports[["airport_id", "Latitude", "Longitude", "TzDatabase"]],
                    left_on = "airport_id",
                    right_on = "airport_id")

nodes.set_index("airport_id", inplace = True)
nodes["TzDatabase"] = nodes["TzDatabase"].str.split('/').str[0]

In [429]:
# Set proper names to columns and index of nodes
nodes.columns = ["Airport", "Latitude", "Longitude", "Continent"]
nodes.index.name = "Airport_id"

In [430]:
# Count number of nodes
N_NODES = nodes.shape[0]

In [431]:
nodes.head()

,Airport,Latitude,Longitude,Continent
Airport_id,,,,
2965,AER,43.449902,39.956600,Europe
2966,ASF,46.283298,48.006302,Europe
2968,CEK,55.305801,61.503300,Asia
4029,DME,55.408798,37.906300,Europe
6156,EGO,50.643799,36.590099,Europe


### 2.2 Create edges

In [432]:
# Create edges from routes between airports
edges = routes[["source_airport_id", "destination_airport_id"]]
edges = edges.replace(to_replace = "\\N", value = np.NaN)
edges.dropna(axis = 0, inplace = True)

# Convert edges id to int
edges["source_airport_id"] = edges["source_airport_id"].apply(int)
edges["destination_airport_id"] = edges["destination_airport_id"].apply(int)

In [433]:
edges.head()

,source_airport_id,destination_airport_id
0,2965,2990
1,2966,2990
2,2966,2962
3,2968,2990
4,2968,4078


In [434]:
# Create mapping from airport id to node id
airport_id2idx = pd.DataFrame(nodes.index.unique())
airport_id2idx = airport_id2idx.reset_index()\
                                .set_index("Airport_id")
airport_id2idx.columns = ["nodes_idx"]

In [435]:
# Add node_ids to edges for adjacency matrix construction
edges = edges.merge(right = airport_id2idx,
                   left_on = "source_airport_id",
                   right_index = True,
                   sort = False)

edges = edges.merge(right = airport_id2idx,
                   left_on = "destination_airport_id",
                   right_index = True,
                   sort = False,
                   suffixes = ["_src", "_dst"])

edges.drop_duplicates(inplace = True)

In [436]:
edges.head()

,source_airport_id,destination_airport_id,nodes_idx_src,nodes_idx_dst
0,2965,2990,0,7
1,2966,2990,1,7
3,2968,2990,2,7
5,4029,2990,3,7
9,6156,2990,4,7


## 3. Create adjacency matrix

### 3.1 Create non-weighted adjacency matrix

In [437]:
# Set all airports with connecting flights as neighbours
# in adjacency matrix
adjacency = np.zeros([N_NODES, N_NODES])
pairs = edges[["nodes_idx_src", "nodes_idx_dst"]].values.T
adjacency[tuple(pairs.tolist())] = 1
adjacency = (adjacency + adjacency.T) > 0

# Eliminate all self ending edges
adjacency = adjacency * (1 - np.identity(N_NODES))

### 3.2 Create laplacian kernel weighted adjacency matrix

The heat kernel weighted adjacency matrix is computed from the laplacian, with each entry having the value $M_{ij}= e^{\frac{d_{ij}}{\sigma}}$

In [438]:
# Set sigma value 
sigma = 1200

In [567]:
# Calculate the distance between airports
dists = edges.merge(nodes[["Airport", "Latitude", "Longitude"]],
                   left_on = "source_airport_id",
                   right_index = True)

dists = dists.merge(nodes[["Airport", "Latitude", "Longitude"]],
                   left_on = "destination_airport_id",
                   right_index = True,
                   suffixes = ("_src", "_dst"))

dists["distance"] = dists.apply(lambda row: distance.geodesic(row[["Latitude_src", "Longitude_src"]].values,
                                                             row[["Latitude_dst", "Longitude_dst"]].values)\
                                                    .km,
                               axis = 1)

In [568]:
# Eliminate self ending edges' entries
dists = dists[dists.apply(lambda row: row["source_airport_id"] != row["destination_airport_id"],
                           axis = 1)]

In [569]:
# Obtain statistics of distance
dists[["distance"]].describe()

,distance
count,36906.000000
mean,1760.875394
std,1946.088833
min,2.832833
25%,571.716646
50%,1156.245634
75%,2092.216138
max,16089.885579


In [ ]:
# Save dists
dists.to_csv("./data/dists.csv", index = False)

In [570]:
# Create heat kernel adjency matrix from distance
laplacian_kernel_adjacency = np.zeros((N_NODES, N_NODES))
pairs = dists[["nodes_idx_src", "nodes_idx_dst"]].values.T
laplacian_kernel_adjacency[tuple(pairs.tolist())] = np.exp(-dists["distance"] / sigma)
laplacian_kernel_adjacency 

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [571]:
# Make the matrix symmetric
unsym_coors = np.array(np.where(laplacian_kernel_adjacency != laplacian_kernel_adjacency.T))

unsym_src = tuple(np.concatenate([unsym_coors, unsym_coors[::-1]], axis = 1).tolist())
unsym_tgt = tuple(np.concatenate([unsym_coors[::-1], unsym_coors], axis = 1).tolist())

laplacian_kernel_adjacency[unsym_src] += laplacian_kernel_adjacency[unsym_tgt]

In [572]:
dists.head()

,source_airport_id,destination_airport_id,nodes_idx_src,nodes_idx_dst,Airport_src,Latitude_src,Longitude_src,Airport_dst,Latitude_dst,Longitude_dst,distance
0,2965,2990,0,7,AER,43.449902,39.956600,KZN,55.606201,49.278702,1507.989680
1,2966,2990,1,7,ASF,46.283298,48.006302,KZN,55.606201,49.278702,1040.943207
3,2968,2990,2,7,CEK,55.305801,61.503300,KZN,55.606201,49.278702,773.126239
5,4029,2990,3,7,DME,55.408798,37.906300,KZN,55.606201,49.278702,718.084202
9,6156,2990,4,7,EGO,50.643799,36.590099,KZN,55.606201,49.278702,1010.815885


In [573]:
nodes.head()

,Airport,Latitude,Longitude,Continent
Airport_id,,,,
2965,AER,43.449902,39.956600,Europe
2966,ASF,46.283298,48.006302,Europe
2968,CEK,55.305801,61.503300,Asia
4029,DME,55.408798,37.906300,Europe
6156,EGO,50.643799,36.590099,Europe


In [574]:
airports[airports["City"] == "Istanbul"]

,airport_id,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TzDatabase,Type,Source,Continent
770,1701,Atatürk International Airport,Istanbul,Turkey,ISL,LTBA,40.976898,28.8146,163,3,E,Europe/Istanbul,airport,OurAirports,Europe
2185,4317,Sabiha Gökçen International Airport,Istanbul,Turkey,SAW,LTFJ,40.898602,29.3092,312,3,E,Europe/Istanbul,airport,OurAirports,Europe


In [575]:
airports.head()

,airport_id,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TzDatabase,Type,Source,Continent
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports,Pacific
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports,Pacific
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports,Pacific
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports,Pacific
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports,Pacific


### 3.2.2 Create largest component matrix for heat kernel adjacency matrix

In [468]:
import networkx as nx

In [577]:
# Create largest connected component graph
G = nx.from_numpy_matrix(laplacian_kernel_adjacency)

largest_cc = max(nx.connected_component_subgraphs(G), key = len)

In [477]:
# Get matrix indices of largest connected component graph
indices = list(largest_cc.nodes)

In [502]:
# Create mapping from idx to airport id
idx2airport_id = airport_id2idx.reset_index().set_index("nodes_idx")

In [496]:
mapping = dict(zip(range(len(indices)), indices))

In [512]:
# Create mapping from largest connected component idx to airport id
new_mapping = dict()

for key in mapping.keys():
    new_mapping[key] = idx2airport_id.loc[mapping[key]].values[0]

In [528]:
np.save("heatker_idx2airport_id.npy", new_mapping)

In [480]:
# Create largest connected component matrix
largest_cc_matrix = heat_kernel_adjacency[indices]
largest_cc_matrix = largest_cc_matrix[:, indices]